In [1]:
using LinearAlgebra, SparseArrays;
using Latexify;

In [2]:
dummy = Dict{String, Array{String}}("a"=>["b","c"], "b"=>["a"], "c"=>["a","d"], "d"=>["b"])

Dict{String, Array{String, N} where N} with 4 entries:
  "c" => ["a", "d"]
  "b" => ["a"]
  "a" => ["b", "c"]
  "d" => ["b"]

In [3]:
input=dummy

Dict{String, Array{String, N} where N} with 4 entries:
  "c" => ["a", "d"]
  "b" => ["a"]
  "a" => ["b", "c"]
  "d" => ["b"]

In [4]:
damping=0.85
function naive_get_matrix(data)
    n=length(dummy)
    encodeNames=String[]
    decodeNames=Dict{String, Int64}()
    i=1
    for (key, value) in data
        push!(encodeNames, key)
        push!(decodeNames, key=>i)
        i=i+1
    end
    M=zeros(Float64, length(dummy), length(dummy))
    for (from, list) in data 
        l=length(list)
        for to in list 
            M[decodeNames[to], decodeNames[from]]=1/l
        end
        M[:,decodeNames[from]].*=damping
        M[:,decodeNames[from]].+=(1-damping)/n
    end
    return (M, encodeNames, decodeNames)
end

naive_get_matrix (generic function with 1 method)

In [5]:
function naive(input)
    M, encode, decode = naive_get_matrix(input)
    Λ, X=eigen(M)
    n=size(Λ,1)
    ranks=X[:, n]
    ranks/=sum(ranks)
    sites=map(r-> (encode[r[1]],Float64(r[2])), enumerate(ranks))
    return sort(sites, by= x->x[2], rev=true)
end

naive (generic function with 1 method)

In [6]:
r=naive(input)

4-element Vector{Tuple{String, Float64}}:
 ("a", 0.37824245632609843)
 ("b", 0.30174695606140833)
 ("c", 0.1982530439385918)
 ("d", 0.12175754367390147)